## 1. Tải Dữ Liệu từ CSV

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import Dataset

def load_csv_dataset(csv_path, text_column, label_column):
    """
    Tải dataset từ file CSV và đổi tên cột.

    Args:
        csv_path (str): Đường dẫn đến file .csv.
        text_column (str): Tên cột chứa văn bản.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        Dataset: Tập dữ liệu đã tải từ file .csv.
    """
    # Tải dữ liệu từ file .csv
    dataset = Dataset.from_csv(csv_path)
    # Đổi tên cột
    dataset = dataset.rename_columns({text_column: "text", label_column: "label"})
    return dataset

# Sử dụng hàm
csv_path = "/content/chatbot_intent_data_v1.csv"             # Đường dẫn file CSV
text_column = "text"       # Cột chứa văn bản
label_column = "intent"        # Cột chứa nhãn

# Tải dataset
dataset = load_csv_dataset(csv_path, text_column, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 28
})
Dataset({
    features: ['text', 'label'],
    num_rows: 10
})


In [3]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["text"], str) or sample["text"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[(3, {'text': None, 'label': 'Im lặng'})]


In [4]:
def preprocess_text(sample):
    if not isinstance(sample["text"], str) or sample["text"].strip() == "":
        sample["text"] = "EMPTY_TEXT"  # Giá trị mặc định
    return sample

dataset = dataset.map(preprocess_text)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [5]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["text"], str) or sample["text"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)


===== Invalid Samples =====
[]


In [6]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.2)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train = train_dataset.select(range(5))  # Lấy 10 mẫu đầu tiên từ train
sample_test = test_dataset.select(range(2))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train)
print("Sample test dataset:", sample_test)

Chia dataset: 22 mẫu train, 6 mẫu test
Train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 22
})
Test dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 6
})
Sample train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 5
})
Sample test dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 2
})


In [7]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn từ dataset
datasets = [train_dataset, test_dataset]
label_to_int = create_label_mapping(datasets)

# Áp dụng chuyển đổi nhãn
train_dataset = train_dataset.map(preprocess_labels, fn_kwargs={"label_to_int": label_to_int})
test_dataset = test_dataset.map(preprocess_labels, fn_kwargs={"label_to_int": label_to_int})

sample_train = train_dataset.select(range(5))  # Lấy 10 mẫu đầu tiên từ train
sample_test = test_dataset.select(range(2))    # Lấy 10 mẫu đầu tiên từ test


# Kiểm tra dữ liệu sau khi chuyển đổi
print("Mẫu train sau chuyển đổi:", train_dataset.select(range(3)))
print("Mẫu test sau chuyển đổi:", test_dataset.select(range(2)))


print("Sample train dataset:", sample_train)


Ánh xạ nhãn: {'Fallback': 0, 'Im lặng': 1, 'Không chắc chắn': 2, 'Từ chối': 3, 'Đồng ý': 4}


Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Mẫu train sau chuyển đổi: Dataset({
    features: ['text', 'label'],
    num_rows: 3
})
Mẫu test sau chuyển đổi: Dataset({
    features: ['text', 'label'],
    num_rows: 2
})
Sample train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 5
})


In [8]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 22
})


## 2. Chuẩn bị tokenizer và token hoá dữ liệu

In [9]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm

# Thiết lập sử dụng GPU cụ thể (ở đây là GPU 1)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Setup CUDA GPU 1

# Định nghĩa class mô hình phân loại ý định sử dụng BERT
class BERTIntentClassification(nn.Module):

    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir="huggingface"):
        """
        Khởi tạo mô hình phân loại ý định sử dụng BERT.
        Args:
            model_name: Tên mô hình BERT được tải từ Hugging Face.
            num_classes: Số lớp (nhãn) cần phân loại.
            dropout_rate: Tỷ lệ dropout trong FFNN để tránh overfitting.
            cache_dir: Thư mục lưu cache mô hình BERT.
        """
        super(BERTIntentClassification, self).__init__()
        # Tải mô hình BERT từ Hugging Face
        self.bert = AutoModel.from_pretrained(model_name, cache_dir=cache_dir)
        # Lấy kích thước hidden của BERT
        hidden_size = self.bert.config.hidden_size
        # Định nghĩa mạng FFNN cho đầu ra
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),  # Lớp fully connected
            nn.LayerNorm(hidden_size),  # Chuẩn hóa layer
            nn.ReLU(),  # Hàm kích hoạt ReLU
            nn.Dropout(dropout_rate),  # Dropout
            nn.Linear(hidden_size, num_classes)  # Lớp fully connected cho đầu ra
        )

    def freeze_bert(self):
        """
        Đóng băng các tham số của BERT, chỉ cập nhật tham số của lớp FFNN.
        """
        for param in self.bert.parameters():
            param.requires_grad = False

    def get_pooling(self, hidden_state, attention_mask):
        """
        Hàm trích xuất biểu diễn (representation) đã được mean pooling từ hidden states của BERT.
        Args:
            hidden_state: Hidden states đầu ra từ BERT.
            attention_mask: Attention mask để xác định các token có giá trị thực.
        Returns:
            pooled_output: Biểu diễn trung bình của toàn bộ chuỗi đầu vào.
        """
        # Lấy last hidden state từ đầu ra của BERT
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Mở rộng attention mask để phù hợp với kích thước hidden state
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask các token padding
            masked_hidden = last_hidden_state * attention_mask

            # Tính tổng các hidden states
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            # Đếm số lượng token thực sự (không phải padding)
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            # Tính trung bình (tổng chia cho số token)
            pooled_output = sum_hidden / count_tokens
        else:
            # Nếu không có attention mask, lấy trung bình tất cả các token
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output

    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass của mô hình.
        Args:
            input_ids: IDs của các token đầu vào.
            attention_mask: Attention mask để xác định token padding.
        Returns:
            logits: Đầu ra raw logits cho mỗi lớp.
        """
        # Lấy hidden states từ BERT
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Trích xuất biểu diễn đã được pooling
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Truyền qua lớp FFNN để tạo logits
        logits = self.ffnn(hidden_state_pooling)

        return logits




In [10]:
from transformers import AutoTokenizer

def initialize_model_and_tokenizer(model_class, model_name="bert-base-uncased", num_classes=2, max_seq_length=512, cache_dir="huggingface"):
    """
    Khởi tạo tokenizer, model và thiết lập thông số.

    Args:
        model_class (class): Lớp model cần khởi tạo.
        model_name (str): Tên model pretrained từ Hugging Face.
        num_classes (int): Số lượng nhãn đầu ra.
        max_seq_length (int): Độ dài tối đa của chuỗi token.
        cache_dir (str): Thư mục lưu trữ cache của mô hình.

    Returns:
        tuple: (tokenizer, model, max_seq_length)
    """
    try:
        # Khởi tạo tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
        print(f"Tokenizer '{model_name}' đã được khởi tạo thành công.")

        # Khởi tạo model
        model = model_class(model_name=model_name, num_classes=num_classes)
        print(f"Model '{model_name}' đã được khởi tạo thành công.")

        # Đóng băng các tham số của BERT
        if hasattr(model, "freeze_bert"):
            model.freeze_bert()
            print("Các tham số của BERT đã được đóng băng.")
        else:
            print("Lớp model không có phương thức 'freeze_bert'.")

        return tokenizer, model, max_seq_length
    except Exception as e:
        print(f"Lỗi khi khởi tạo model hoặc tokenizer: {e}")
        raise


# Khởi tạo tokenizer và model
tokenizer, model, max_seq_length = initialize_model_and_tokenizer(
    model_class=BERTIntentClassification,
    model_name="bert-base-uncased",
    num_classes=5,
    max_seq_length=512,
    cache_dir="huggingface"
)

# # Kiểm tra kết quả
# print(tokenizer)
# print(model)
# print(f"Max sequence length: {max_seq_length}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer 'bert-base-uncased' đã được khởi tạo thành công.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model 'bert-base-uncased' đã được khởi tạo thành công.
Các tham số của BERT đã được đóng băng.


## Testing 1 câu trước huấn luyện models

In [11]:
print("===== Tokenizer Info =====")
print("Vocab size:", tokenizer.vocab_size)
print("Max model input size:", tokenizer.model_max_length)
print("Special tokens:", tokenizer.special_tokens_map)


===== Tokenizer Info =====
Vocab size: 30522
Max model input size: 512
Special tokens: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


In [12]:
print("\n===== Model Info =====")
print("Model architecture:", model.bert.config.architectures)
print("Hidden size:", model.bert.config.hidden_size)
print("Number of classes:", model.ffnn[-1].out_features)  # Lớp đầu ra của FFNN



===== Model Info =====
Model architecture: ['BertForMaskedLM']
Hidden size: 768
Number of classes: 5


In [13]:
print("\n===== Thử Tokenizer với một câu ví dụ=====")
test_sentence = "Hello, how are you?"
encoded = tokenizer(test_sentence, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")

print("\n===== Tokenizer Output =====")
print("Input IDs:", encoded["input_ids"])
print("Attention Mask:", encoded["attention_mask"])



===== Thử Tokenizer với một câu ví dụ=====

===== Tokenizer Output =====
Input IDs: tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


In [14]:
print("\n===== Forward Pass Output =====")
with torch.no_grad():  # Không cần tính gradient
    input_ids = encoded["input_ids"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    attention_mask = encoded["attention_mask"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    logits = model(input_ids=input_ids, attention_mask=attention_mask)

print("Logits shape:", logits.shape)
print("Logits:", logits)



===== Forward Pass Output =====
Logits shape: torch.Size([1, 5])
Logits: tensor([[-0.2873,  0.0726,  0.1078, -0.5539, -0.3925]])


In [15]:
# Tính xác suất với softmax
softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(logits)

print("Probabilities:", probabilities)
print("Predicted class:", torch.argmax(probabilities).item())


Probabilities: tensor([[0.1791, 0.2567, 0.2659, 0.1372, 0.1612]])
Predicted class: 2


In [16]:
print("\n===== Compute Loss =====")
labels = torch.tensor([1]).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Nhãn mẫu
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)
print("Loss:", loss.item())



===== Compute Loss =====
Loss: 1.3599333763122559


In [17]:
def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("text"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs

In [18]:
train_dataset
print("===== Train Dataset =====")
for sample in train_dataset:
    print(sample)


===== Train Dataset =====
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}
{'text': 'Không, tôi không thấy cần thiết.', 'label': 3}
{'text': 'Chắc chắn rồi, để tôi làm.', 'label': 4}
{'text': 'Không, tôi không đồng ý.', 'label': 3}
{'text': 'Hoàn toàn chính xác, để tôi làm.', 'label': 4}
{'text': 'Im lặng trong 10 giây.', 'label': 1}
{'text': 'Không, tôi nghĩ là không cần.', 'label': 3}
{'text': 'Không có gì để nói cả.', 'label': 1}
{'text': 'EMPTY_TEXT', 'label': 1}
{'text': 'Vâng, tôi muốn cho bạn xem bức tranh.', 'label': 4}
{'text': 'Vâng, tôi đồng ý với bạn.', 'label': 4}
{'text': 'Xin lỗi, tôi không chắc bạn đang nói gì.', 'label': 0}
{'text': 'Tôi nghĩ tôi cần thêm thời gian để xem xét.', 'label': 2}
{'text': 'Không, điều đó không đúng.', 'label': 3}
{'text': 'Không có phản hồi nào trong 5 giây.', 'label': 1}
{'text': 'Không có âm thanh nào phát ra.', 'label': 1}
{'text': 'Tôi không biết.', 'label': 2}
{'text': 'Xin lỗi, tôi không hiểu bạn đang nói gì.', 'label': 0}
{'text': '

Login Wandb

In [19]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [20]:
!pip install python-dotenv

In [21]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [22]:
# In giá trị của API key
wandb_api_key = os.getenv("WANDB_API_KEY")
if wandb_api_key:
    print(f"API Key Loaded: {wandb_api_key[:5]}...")  # Chỉ in 5 ký tự đầu để kiểm tra
else:
    print("API Key not found in environment variables.")


API Key Loaded: c8767...


In [23]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Cài tham số models

In [24]:
# Custom Trainer class để điều chỉnh cách tính loss
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Hàm tính toán loss được sử dụng bởi Trainer.
        Args:
            model: Mô hình đang huấn luyện.
            inputs: Dữ liệu đầu vào, bao gồm input_ids, attention_mask, và labels.
            return_outputs: Có trả về outputs hay không.
        Returns:
            loss: Giá trị loss đã tính toán.
        """
        # Tách labels từ inputs (nếu có)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() để tính toán loss
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và lấy đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo đầu ra là logits
        logits = outputs

        # Tính toán loss dựa trên logits và labels
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


In [25]:
# # Bước 6: Cài đặt tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="./results",          # Thư mục lưu kết quả
#     eval_strategy="epoch",    # Đánh giá sau mỗi epoch
#     learning_rate=2e-4,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     num_train_epochs=50,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=None,
#     logging_strategy = "epoch",
#     save_strategy="epoch",          # Lưu trọng số sau mỗi epoch
#     save_total_limit=3,
# )

# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,  # Ghi log sau mỗi 10 bước
    logging_strategy="steps"  # Đảm bảo ghi log theo steps
)




In [27]:
# Bước 7: Tạo Trainer
trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=sample_train,
    eval_dataset=sample_test,
    tokenizer=tokenizer,
    data_collator = collate_fn,
)

# Bước 8: Huấn luyện
trainer.train()

<ipython-input-27-ba8ccdf04d93>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerCustom.__init__`. Use `processing_class` instead.
  trainer = TrainerCustom(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# Bước 9: Đánh giá trên tập kiểm tra
trainer.evaluate()